# <center>实验3 决策树算法实践 </center>

# 一、实验目的

通过实验，达到以下目的：

- 使学生加深对机器学习过程的理解；
- 使学生理解信息熵与信息增益的计算方法；
- 使学生掌握决策树的构建方法和程序实现；
- 使学生能够熟练应用sklearn库实现基于决策树方法的分类预测。

# 二、实验内容


## 决策树（Decision Tree）

决策树是一种常见的机器学习方法。常用于分类任务。

决策树，顾名思义，是基于树结构来进行决策的，这是人类在面临决策问题时的一种很自然的处理机制。


## 基本流程

决策树的主要优势在于决策过程很容易理解。利用决策树形成的判断过程，同富有经验的领域专家几乎实现相同的。

下面我们分析一下决策树的基本处理流程。

以二分类任务为例，我们希望从给定训练数据集学得一个模型，用于对新示例进行分类，这个对样本进行分类的任务，可以看作对“当前样本属于正类么？”这个问题的“决策”或“判定”过程。

![决策树示例3](images\dt\西瓜问题决策树.png)

- 决策过程的最终结论（树的叶子），对应了我们希望的判定结果；

- 决策过程中提出的每个判定问题，都是对某个属性的“测试”；

- 每个测试的结果，要么导出最终结论，要么导出进一步的判定问题，其考虑范围是在上一次决策结果的限定范围之内；

一般地，一颗决策树，包含一个根节点，若干个内部结点，和若干个叶节点。

- 叶节点，对应决策结果；

- 其它节点，对应一个属性测试。

- 每个节点，包含的样本集合，根据属性测试的结果被划分到子节点中；

- 根节点，包含样本全集；

- 从根节点到每个叶节点的路径，对应了一个判定测试序列。

**决策树学习的目的，是为了产生一棵泛化能力强的决策树，基本流程遵循简单而且直观的“分而治之（divide-and-conquer）”策略。**




## 决策树的构造

如何根据训练集数据，建立决策树呢？

结合上面的基本过程，我们知道，决策树的每个节点对应了一个属性测试，在数据集中往往有一些属性是“好属性”，使用它们做测试，能够有“正确”的分类结果。


### 构造决策树的伪代码

输入1：训练集$D={(x_1,y_1),(x_2,y_2),...,(x_m,y_m)}$   
输入2：属性集$A = {a_1,a_2,...,a_d}$  
过程：函数 TreeGenerate(D,A)


生成结点；  

if D 中样本全属于同一类别C :  
&ensp;&ensp;&ensp;&ensp;将node标记为C类叶结点；  

if A = 空集 or D 中样本在 A 上取值相同 :    
&ensp;&ensp;&ensp;&ensp; 将 node 标记为叶结点，其类别标记为 D 中样本数最多的类；  
&ensp;&ensp;&ensp;&ensp; return；
    
从 A 中选择最优划分属性 $a_*$;  
for $a_*$ 的每一个值 $a_*^v$ :    
&ensp;&ensp;&ensp;&ensp;为node生成一个分支；  
&ensp;&ensp;&ensp;&ensp;令$D_v$表示D中在$a_*$上取值为$a_*^v$的样本子集；  
&ensp;&ensp;&ensp;&ensp;if $D_v$ 为空：  
        &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;将分支结点标记为叶节点，其类别标记为D中样本最多的类；  
        &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;return    
&ensp;&ensp;&ensp;&ensp;else：  
        &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;以$TreeGenerate(D_v,A \ {a_*})$ 为分支结点；  
   
输出： 以node为根节点的一棵决策树。

此时，我们的问题聚焦到了“数据集上哪些特征在划分数据分类时，起决定性作用？”，换句话说，**“如何选择最优划分属性？”**

### 划分选择

为了找到最好的决策属性，我们需要评估每个属性。那么以何种标准来评估？

一般而言，随着划分过程不断进行，我们希望决策树的分支结点所包含的样本，尽可能属于同一类别，即结点的“纯度”（purity）越来越高。

如果可以度量，那么我们可以根据纯度来评估每个属性，找到最好的决策属性。

如何度量样本集合的纯度呢？

### 信息熵

“信息熵”（information entropy）是度量样本集合纯度最常用的一种指标。

假定当前样本集合D中第k类样本所占的比例为$p_k(k = 1,2,...,|y|)$，则$D$的信息熵定义为：

$Ent(D) = - \sum_{k=1}^{|y|}p_k \log_{2}p_k$    ——式（1）

Ent(D)的值越小，则D的纯度越高。熵在信息论中代表随机变量不确定度的度量，熵值越大，不确定性越高。

> 对熵的直观解释，可以参考https://blog.csdn.net/qq_39521554/article/details/79078917

假定离散属性a有V个可能的取值${a^1,a^2,...,a^v}$，若使用a来对样本集D进行划分，则会产生V个分支结点。

其中，第$v$个分支结点包含了D中所有在属性a上取值为$a^v$的样本，记为$D^v$。

根据式（1），计算出$D^v$的信息熵。

即当前集合$D^v$，设第k类样本所占的比例为$p_k^v(k = 1,2,...,|y^v|)$

$Ent(D^v) = - \sum_{k=1}^{|y^v|}p_k^v \log_{2}p_k^v$ 


下面给出计算$D^v$的香农信息熵的python程序代码

In [2]:
"""计算香农信息熵的python程序代码
"""
from math import log
from sklearn import neighbors,datasets
import pandas as pd

def calcShannonEnt(dataFrame):
    """
    功能：根据分类标记，计算某数据集的信息熵。
    输入：dataFrame，使用pandas.seriers类型给出的含有标记的数据集，标记信息为最后一列
    输出：shannonEnt，数据集按当前标记分类结果的信息熵值
    
    """  
    numEntries = dataFrame.shape[0] #s数据集示例数
    
    labelCounts = {} #定义字典，键为分类标记名，值为标记的计数值
    labelCounts.update(dataFrame.iloc[:,-1].value_counts())#为字典赋值,认为数据集最后一列为label
    
    shannonEnt = 0.0  # 设置香农信息熵初值为0.0
    for key in labelCounts:
        # 按公式求信息熵值
        prob = float(labelCounts[key])/numEntries
        
        shannonEnt -= prob * log(prob,2)    # 求以2为底的对数。
    return shannonEnt

def calcDatingDataSetEnt():
    dataframe = pd.read_csv("data\dating\datingTestSet.txt",header=None,sep='\t',names=['年飞机里程','周冰淇淋升数','游戏耗时比','心仪程度'])
    entropy = calcShannonEnt(dataframe)
    print("当前数据集的香农信息熵：%f"  % entropy)

calcDatingDataSetEnt()


当前数据集的香农信息熵：1.584702


上面的例子中计算得到的信息熵差别不大，我们再看一个例子。

这个例子是美国大选投票数据集，基本情况如下图所示：

![选举数据示例](images/dt/voteexample.png)

这个例子将计算一个投票数据集的熵值

In [3]:
def calcAllVoteFeaturesEnt():
    dataframe = pd.read_csv("data/vote/VoteTraining-cn.csv",header=0,sep=',')
    
    entropy = calcShannonEnt(dataframe)
    print("信息熵：%f"  % entropy)

calcAllVoteFeaturesEnt()

信息熵：0.996566


### 信息增益

假定离散属性a有V个可能的取值${a^1,a^2,...,a^v}$，若使用a对样本集D进行划分，则会产生V个分支结点，其中第v个分支结点包含了D中所有在属性a上取值为$a^v$的样本，记为$D^v$。

我们可以根据式（1）计算出$D^v$的信息熵，再考虑到不同的分支结点所包含的样本数不同，给分支结点赋予权重$\frac{|D^v|}{|D|}$，即样本数越多的分支终点的影响越大，于是可计算出用属性a对样本集D进行划分所获得的“信息增益”（information gain)。

$Gain(D,a) = Ent(D) - \sum_{v=1}^{V}\frac{|D^v|}{|D|}Ent(D^v)$  ——式（2）

一般而言，信息增益越大，则意味着使用属性a来进行划分所获得的“纯度提升”越大。

因此，我们可用信息增益来进行决策树的划分属性选择，即在选择“最好的”属性进行决策。

$a_* = argmax_{a \in A}Gain(D,a)$

著名的ID3决策树学习算法【Quinlan，1986】就是以信息增益为准则来划分属性。

下面程序的例子是以美国大选的投票数据样本集为例，选择最好划分属性的代码：

In [4]:
def calcInforGain(df,aFeature):
    """
    功能：按照上述公式计算用属性aFeature对样本集dataframe进行划分的信息增益。
    输入：数据集dataFrame；
          划分属性名aFeature；
    输出：（最好划分属性名称，最大信息增益值）
    """
    # 统计数据集的样本数量
    totalsampleCount = df.shape[0]
    # 统计属性a各值对应的样本数量
    sampleCounts = {} 
    sampleCounts.update(df[aFeature].value_counts())
    #print(sampleCounts)
    infogain = calcShannonEnt(df)
    for key,value in sampleCounts.items():
        subdf = df[df[aFeature] == key]
        infogain -= subdf.shape[0]/ totalsampleCount * calcShannonEnt(subdf)
    return infogain

def getBestDivideFeature(df):    
    featureInfoGains = {}    
    for colname in df.columns[:-1]:
        # 对非标记属性，计算其信息增益，标记属性为dataframe中最后一列
        infogain = calcInforGain(df,colname)
        featureInfoGains[colname] = infogain
    # 对已计算增益的结果进行排序
    bestFeature = sorted(featureInfoGains.items(),key =lambda item:item[1],reverse=True)[0]
    #print(featureInfoGains)
    return bestFeature

#dataframe = pd.read_csv("data/vote/VoteTraining-cn.csv",header=0,sep=',')        
#getBestDivideFeature(dataframe)    
dataframe = pd.read_csv("data/maloon/maloon2.txt",header=0,sep=',')        
getBestDivideFeature(dataframe.iloc[:,1:])    

('纹理', 0.3805918973682686)

根据计算信息增益，发现在投票数据集中，“医师费用冻结”属性对数据集进行划分的信息增益最大，于是我们将选择它作为划分属性。

划分结果如下：


In [5]:
dataframe = pd.read_csv("data/vote/VoteTraining-cn.csv",header=0,sep=',')
dict = {}
dict.update(dataframe["医师费用冻结"].value_counts())
subDList = {}
for colValue in dict.keys():
    subDList[colValue] = dataframe[dataframe["医师费用冻结"]==colValue]

print("按\"医师费用冻结\"进行划分:")
for i in subDList.items():
    print("    {}类的节点数为{}".format(i[0],i[1].shape[0]))

按"医师费用冻结"进行划分:
    n类的节点数为119
    y类的节点数为113


然后，决策树学习算法将对每个分支结点做进一步划分。

如上例中，通过“医师费用冻结”进行划分后，需要再次计算“最好划分属性”，对各分支结点所含数据集进行划分。

### 构建决策树的程序

选择最佳属性的过程一般需要多次，对每个分支结点进行上述操作，直至将所有属性都使用完毕。这样一棵决策树就建立起来了。

但是，大多数用户不需要这样的决策树，而是需要那种只通过3~5个属性值就能够进行分类的决策树。

这时需要由用户事先指定“树的深度”这个超参来构建决策树。

下面，根据上文中构造决策树的伪代码，我们编写决策树构建函数：

In [6]:
import pandas as pd

class Node:
    """决策树结点类"""
    def __init__(self):
        self._type = None
        self._label = None
        self._samples = pd.DataFrame()
        self._children = {}        
    
    @property    
    def type(self):
        return self._type
    
    @type.setter
    def setType(self,type):
        self._type = type
    
    @property    
    def label(self):
        return self._label
    
    @label.setter
    def setLabel(self,label):
        self._label = label
        
    @property
    def samples(self):
        return self._samples
    
    @samples.setter
    def setSamples(self,samples):
        if isinstance(samples, pd.DataFrame):
            self._samples = samples
    
    @property    
    def children(self):
        return self._children
    

    def addChildren(self,key,children):
        self._children[key] = children

    def getChildrenTypeList(self):
        cl = []
        for k,v in self._children.items():
            cl.append(v.type + '['+k+']')
        return cl
    
    def __str__(self):        
        return "Type:{},Label:{},Samples:{},Children:{} ".format(
                self._type,self._label,
                self._samples.index.tolist(),
                self.getChildrenTypeList())
def bfs(rootNode,depth = 0):
    print("{}{}".format('\t'*depth,rootNode))
    #print("Type:{},Label:{},Samples:{}".format(rootNode.type,rootNode.label,rootNode.samples.index.tolist()))
    for k,v in rootNode.children.items():
        bfs(v,depth + 1)
    return depth
        
              
    

下面的代码用于测试上述类定义

In [7]:
node = Node()
node.setType = 'root' 
child = Node()
node.addChildren('a=1',child)
child.setType = 'middle'
child2 = Node()
node.addChildren('a=2',child2)
child2.setType = 'middle'
ss = Node()
ss.setType = 'leaf'
ss1 = Node()
ss1.setType = 'leaf'
child.addChildren('b=1',ss)
child.addChildren('b=2',ss1 )
print(node)
bfs(node)

Type:root,Label:None,Samples:[],Children:['middle[a=1]', 'middle[a=2]'] 
Type:root,Label:None,Samples:[],Children:['middle[a=1]', 'middle[a=2]'] 
	Type:middle,Label:None,Samples:[],Children:['leaf[b=1]', 'leaf[b=2]'] 
		Type:leaf,Label:None,Samples:[],Children:[] 
		Type:leaf,Label:None,Samples:[],Children:[] 
	Type:middle,Label:None,Samples:[],Children:[] 


0

In [2]:
import pandas as pd

def createDT(dataFrame,depth):
    """
    功能：根据有标记数据集dataFrame，构建深度为depth的决策树
    输入：训练集 𝐷 = dataFrame ，属性名为dataframe的第0行
          树的深度为depth，默认值为3
    输出：一棵以嵌套字典表示的决策树  

    """
    # 生成结点
    node = Node()
         
    if len(dataFrame.iloc[:,-1].value_counts()) == 1:
        #若D 中样本全属于同一类别，则将node标记为C类叶结点
        node.setType = 'Leaf'
        node.setLabel = dataFrame.iloc[0,-1]
        node.setSamples = dataFrame    
        return node

    if len(dataFrame.iloc[:,:-1]) == 0 :
        # A = 空集 :
        # 将 node 标记为叶结点，其类别标记为 D 中样本数最多的类；
        # 注意，因为可能存在未知因素，会存在A上取值相同，但分类不同的示例。        
        tempdict = {}
        tempdict.update(dataFrame.iloc[:,-1].value_counts())  
        if tempdict:
            label = sorted(tempdict,key =lambda item:item[1],reverse=True)[0]  
        else:
            label = ''
        node.setType = 'Leaf' 
        node.setLabel = label
        node.setSamples = dataFrame
        return node
    
    valueIsUnique = False
   
    for feature in dataFrame.columns[:-1]:    
        if len(dataFrame[feature].value_counts()) > 1:
            # 属性集任一个属性的取值不唯一，就跳出
            break
        valueIsUnique = True
    if valueIsUnique :
        # D 中样本在 A 上取值相同 :
        # 将 node 标记为叶结点，其类别标记为 D 中样本数最多的类；
        # 注意，因为可能存在未知因素，会存在A上取值相同，但分类不同的示例。        
        tempdict = {}
        tempdict.update(dataFrame.iloc[:,-1].value_counts())        
        label = sorted(tempdict,key =lambda item:item[1],reverse=True)[0]        
        node.setType = 'Leaf' 
        node.setLabel = label
        node.setSamples = dataFrame      
        return node
  
    #从 A 中选择最优划分属性  𝑎∗ 
    
    aFeature = getBestDivideFeature(dataFrame)[0]    
    #print("  使用{}作为划分依据".format(aFeature))
    node.setType = "据\"{}\"划分".format(aFeature)
    node.setLabel = '未定'
    node.setSamples = dataFrame
    
    dict = {}
    dict.update(dataFrame[aFeature].value_counts())
    #print("dict内容：{}".format(dict))
    for colValue in dict.keys():
        keyname = aFeature+'='+colValue
        #对𝑎∗ 的每一个值  𝑎𝑣∗ ，为node生成一个分支 
        aBranchNode = Node()                      
        #令 𝐷𝑣 表示D中在 𝑎∗ 上取值为 𝑎𝑣∗ 的样本子集；
        dv = dataFrame[dataFrame[aFeature]==colValue]         
        if  dv.empty:
            # 若dv为空,将分支结点标记为叶节点，其类别标记为D中样本最多的类；
            tempdict = {}
            tempdict.update(dataFrame.iloc[:,-1].value_counts())        
            label = sorted(tempdict,key =lambda item:item[1],reverse=True)[0]              
            aBranchNode.setType = 'Leaf' 
            aBranchNode.setLabel = label
            aBranchNode.setSamples = dv
            # 将aBranchNode列为node的子节点           
            node.addChildren(keyname,aBranchNode)              
        else:    
            # 去除数据集Dv 中aFeature列后的样本
            subDv = dv.drop([aFeature],axis =1)            
            if depth == 0:
                #若当前树的深度已经达到要求，则将当前分支节点的其类别标记为D中样本最多的类
                tempdict = {}
                tempdict.update(subDv.iloc[:,-1].value_counts())        
                label = sorted(tempdict,key =lambda item:item[1],reverse=True)[0]              
                aBranchNode.setType = 'Leaf' 
                aBranchNode.setLabel = label    
                aBranchNode.setSamples = subDv
                # 将aBranchNode列为node的子节点                
                node.addChildren(keyname,aBranchNode)     
                continue
            # 以 createDT(subDv)  为分支结点；
            node.addChildren(keyname,createDT(subDv,depth-1))
    return node


下面我们以西瓜数据集进行测试

In [ ]:
#dataframe = pd.read_csv("data/vote/votesimple.txt",header=0,sep=',')    
df = pd.read_csv("data1/maloon/maloon2.txt",header=0,sep=',')
d = df.iloc[:,1:]
dtree = createDT(d,depth = 5)
print('决策树：')
bfs(dtree)

## sklearn库决策树分类器应用

上面的程序是构建决策树的示例。在实际应用中，我们往往使用较为成熟的第三方工具sklearn。

下面的示例中，使用sklearn.tree 中的DecisionTreeClassifier对投票数据进行预测。

In [1]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import pandas as pd

def decisionTree(list):
    dataframe = pd.read_csv("data/vote/VoteTraining-cn.csv",header=0,sep=',')
    #data.describe()
    X = dataframe.iloc[:,:-1] #存放训练样本中无标记的数据
    X1 = pd.DataFrame() # sklearn的算法分类器大多只处理数值型矩阵，X1将存放数值化的样本
    for column in X.columns:
        X1[column] = X[column].apply(lambda x:1 if x=='y' else 2)
        
    y = dataframe.iloc[:,-1] # 存放标记    
    y1 = y.apply(lambda x:1 if x == "republican" else 2) # 对标记进行数值化
        
    # 使用决策树模型对X,y进行拟合，即生成决策树分类器
    clf = DecisionTreeClassifier(max_depth=3)
    clf.fit(X1, y1)
    # 对输入的list按上面生成的决策树分类器进行批量预测
    predict = clf.predict([list])     
    return predict[0]

def predict():
    dataframe = pd.read_csv("data/vote/Vote.csv",header=None,sep=',')    
    for index,row in dataframe.iterrows():
        #print(row.tolist()[:-1])
        row = row.apply(lambda x:1 if x=='y' else 2)
        result = decisionTree(row.tolist()[:-1])
        if result == 1:
            print("republican")
        else:
            print("democrat")
        
predict()

democrat
democrat
democrat
democrat
democrat
republican
republican


### 理解sklearn的决策树分类器

下面我们通过分析程序来理解上述决策树的基本情况：

In [81]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import pandas as pd


dataframe = pd.read_csv("data/vote/VoteTraining-cn.csv",header=0,sep=',')
#data.describe()
X = dataframe.iloc[:,:-1] #存放训练样本中无标记的数据
X1 = pd.DataFrame() # sklearn的算法分类器大多只处理数值型矩阵，X1将存放数值化的样本
for column in X.columns:
    X1[column] = X[column].apply(lambda x:1 if x=='y' else 2)
y = dataframe.iloc[:,-1] # 存放标记    
y1 = y.apply(lambda x:1 if x == "republican" else 2) # 对标记进行数值化

# 使用决策树模型对X,y进行拟合，即生成决策树分类器
clf = DecisionTreeClassifier(max_depth=3)
clf.fit(X1, y1)

n_nodes = clf.tree_.node_count
children_left = clf.tree_.children_left
children_right = clf.tree_.children_right
feature = clf.tree_.feature
threshold = clf.tree_.threshold
    
print("n_nodes={}".format(n_nodes))
print("children_left={}".format(children_left))
print("children_right={}".format(children_right))
print("feature={}".format(feature))
print("threshold={}".format(threshold))

# The tree structure can be traversed to compute various properties such
# as the depth of each node and whether or not it is a leaf.
node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, -1)]  # seed is the root node id and its parent depth
while len(stack) > 0:
    node_id, parent_depth = stack.pop()
    node_depth[node_id] = parent_depth + 1

    # If we have a test node
    if (children_left[node_id] != children_right[node_id]):
        stack.append((children_left[node_id], parent_depth + 1))
        stack.append((children_right[node_id], parent_depth + 1))
    else:
        is_leaves[node_id] = True

print("The binary tree structure has %s nodes and has "
      "the following tree structure:"
      % n_nodes)
for i in range(n_nodes):
    if is_leaves[i]:
        print("%snode=%s leaf node." % (node_depth[i] * "\t", i))
    else:
        print("%snode=%s test node: go to node %s if X[:, %s] <= %s else to "
              "node %s."
              % (node_depth[i] * "\t",
                 i,
                 children_left[i],
                 feature[i],
                 threshold[i],
                 children_right[i],
                 ))
print()

n_nodes=11
children_left=[ 1  2  3 -1 -1 -1  7 -1  9 -1 -1]
children_right=[ 6  5  4 -1 -1 -1  8 -1 10 -1 -1]
feature=[ 3 10  8 -2 -2 -2  2 -2  5 -2 -2]
threshold=[ 1.5  1.5  1.5 -2.  -2.  -2.   1.5 -2.   1.5 -2.  -2. ]
The binary tree structure has 11 nodes and has the following tree structure:
node=0 test node: go to node 1 if X[:, 3] <= 1.5 else to node 6.
	node=1 test node: go to node 2 if X[:, 10] <= 1.5 else to node 5.
		node=2 test node: go to node 3 if X[:, 8] <= 1.5 else to node 4.
			node=3 leaf node.
			node=4 leaf node.
		node=5 leaf node.
	node=6 test node: go to node 7 if X[:, 2] <= 1.5 else to node 8.
		node=7 leaf node.
		node=8 test node: go to node 9 if X[:, 5] <= 1.5 else to node 10.
			node=9 leaf node.
			node=10 leaf node.



# 三、实验要求

1. 学生应当能够在教师的指导下理解上述处理过程和python代码实现；
2. 参考实验内容完成一个自选新数据集的分类预测；
3. 根据实验过程与结果编写实验报告。